In [ ]:
import numpy as np
import pandas as pd
import xarray as xr


## Collect predictions; format and save coSIF datasets

In [ ]:
format_time = lambda ds: ds.expand_dims(month=[pd.to_datetime(ds["time"].data)], axis=-1)
ds_predictions = xr.open_mfdataset(
    "../data/production/*riging_results_*.nc4", preprocess=format_time
)
ds_predictions


In [ ]:
months = ["202102", "202104", "202107", "202110"]

for m in months:
    ds = ds_predictions.sel(month=m)[["predictions", "rmspe"]].rename(
        {"predictions": "cosif_prediction", "rmspe": "cosif_rmspe"}
    )
    display(ds)
    # .to_netcdf("../data/production/coSIF_{m}.nc4")


## Calculate average standard error ratios (compared to the data)

In [ ]:
with xr.open_dataset(
    "../data/production/OCO2_005deg_monthly_north_america_with_basis.nc4"
) as ds:
    ds_data = ds.rename(time="month").sel(month=ds_predictions.month)

ds_data


In [ ]:
# NOTE: groupby month when we have all predictions again

# NOTE: "sif_var" is a misnomer, it is actually the standard error
df_se = (
    xr.merge([ds_predictions["mspe"], ds_data["sif_var"]])
    .squeeze()
    .astype(np.float64)
    .to_dataframe()
    .reset_index()
    .drop(columns=["month", "time"])
    .dropna(subset="sif_var")
)
df_se["rmspe"] = np.sqrt(df_se["mspe"])
df_se["se_ratio"] = df_se["rmspe"] / df_se["sif_var"]
df_se["se_ratio"].mean()
